In [1]:
import numpy as np
import pandas as pd
import scipy as sp

from scipy.sparse import random, csr_matrix

# 1 Framework

**Recommendation Systems (RS)** are software systems that recommend items to users, that they might like.

We start by learning the main components of an RS.

![Recommender Sytems Framework](./BLU10 - Non-personalised recommender systems/media/recommender_systems_framework.png)

*Fig.1 - RS framework with a community, plus the basic and extended models.*

We refer back to this framework frequently throughout the specialization, but for now, let's drill down into each component.

## 1.1 Users

The *consumers* or people, denoted by $U = \{u_0, u_1, ..., u_m\}$.

We reserve the indexing letters $u$ and $v$ to denote generic individual users.

## 1.2 Items

The *products* or things, a set $I = \{i_0, i_1, ..., i_n\}$. 

The indexing letters for letters for items are $i$, $j$ and $l$.

## 1.3 Community

The space of all possible ($u$, $i$) pairs is given by $U \times I = \{(u_0, i_0), (u_0, i_1), ..., (u_0, i_n), ..., (u_m, i_n)\}$.

The resulting set $U \times I$ contains $\left\vert{U}\right\vert * \left\vert{I}\right\vert = m * n$ possible combinations.



## 1.4 Ratings

We write the set $R = \{r_{u_0, i_0}, ..., r_{u_m, i_n}\}$, where each rating $r_{u, i}$ corresponds to a user-item pair $(u, i) \in U \times I$.

We use $R'$ for the set of all *recorded* ratings, such as:

* Any user $u \in U$ can make no more than one rating $r_{u, i}$ for a particular item $i \in I$.
* Any user $u \in U$ is free to rate, or not, any number of items $i \in I$, including none.

Formally, the set of recorded ratings (**existing ratings**) $R' \subseteq R$ (**all possible ratings**). 
<br>

#### Explicit and implicit ratings

Feedback is said to be explicit when provided by the user and implicit if inferred based on user actions (e.g., clicks).  

Implicit feedback usually takes the form of unary data.

#### Rating scale

We write $S$ the set of possible ratings. For example, in 1-5 stars rating system $r_{u, i} \in S = \{1, 2, 3, 4, 5\}$.

| Type of data    | Description                          | Rating scale (examples) | Explicit/Implicit |  
|-----------------|--------------------------------------|-------------------------|-------------------|
| Numeric         | Continuous ratings                   | $S = [1, 5]$            | Explicit          |
| Ordinal         | Ordered categories                   | $S = \{1, 2, 3, 4, 5\}$ | Explicit          |
| Binary          | Good or bad  (e.g., Upvote/Downvote) | $S = \{-1, 1\}$         | Explicit          |
| Unary           | User action  (e.g., Click, Purchase) | $S = \{1\}$             | Implicit          |
*Table 1: Different types of data and rating scales*


## 1.5 Ratings matrix 

We represent the set $R = \{r_{u_0, i_0}, ..., r_{u_m, i_n}\}$ as a $U \times I$  matrix - the **ratings matrix** -, where the values are the ratings $r_{u, i}$, if they exist:

$$\begin{bmatrix}r_{u_0, i_0} & r_{u_0, i_1} & ... & r_{u_0, i_n}\\ r_{u_1, i_0} & r_{u_1, i_1} & ... & r_{u_1, i_n}\\... & ... & ... & ...\\ r_{u_m, i_0} & r_{u_m, i_1} & ... & r_{u_m, i_n}\end{bmatrix}$$

We represent not recorded ratings $r_{u, i} \notin R'$ as zeros or missing values, enforcing the $U \times I$ shape.


#### Example of rating matrix

Consider the following ratings matrix $R$, with $S = \{1, 2, 3, 4, 5\}$:

$$\begin{bmatrix}1 &  & 2\\ 1 & 5 & \\  & 2 & 1\end{bmatrix}$$


## 1.6 Important subsets

The subset of users that have rated an item $i \in I$ is given by $U_i \subseteq U$. 

We write $I_u \subseteq I$ the subset of items rated by user $u \in U$.

The subset items that have been rated by users $u, v \in U$ is $I_{u, v} \subseteq I$. 

Finally, $U_{i, j} \subseteq U$ is used for the subset of users that rated both items $i, j \in I$.


---
---

## 1.7 Profiles

**Profiles** are a collection of objects (users or items, in our framework) and their **attributes**.

Consider the set of attributes $A = \{a_0, ..., a_r\}$, where $r \in \mathbb{N_0}$. Let be $A$ an arbitrary set of **item attributes**.

We can define profiles $P = \{p_{i_0, a_0}, ..., p_{i_n, a_r}\}$, where $(i, a) \in I \times A$ and values $p_{i, a}$ indicate the presence of $a$ in $i$.

Let be $B$ be an arbitrary set of user characteristics, and we can apply the same reasoning to build **user profiles**.

Typically, item profiles contain information about the content of the items, and user profiles are more focused on demographics.


## 1.8 User model

As introduced above, RS are in the business of matching users and items.

Sometimes, it's convenient to have user and item profiles in the same attribute space $A$.

The user model $M$, defines $M = \{m_{u_0, a_0}, ..., m_{u_m, a_r}\}$, for $(u, a) \in U \times A$, where $A$ is the set of item attributes.

# 2 Recommendations

**The whole objective of Recommender Systems is to fill in the blanks in our ratings matrix** ($r_{u, i} \notin R'$) and return the best possible items to the user.

Throughout the course, we learn different ways to predict unseen ratings.

### Prediction step

**The RS core computation is to predict the utility of unseen items**.

That is, for the user $u$ we need to predict $r_{u, i}$ for each item $i \in I \setminus I_u$.  
<br>
<br>


In mathematical language, we want to learn a function $f$ that maps user-item pairs into ratings $f : U \times I \to S$ given by $\hat{r}_{u, i} = f(u, i)$ where $S$ is the set of possible ratings.

Once we have it, there two main types of recommendations: **top-$N$** and **best-item**.

### Top-*K* items

For a given user $u \in U$, we need a set of predictions $\hat{R}_u = \{f(u, i) : (u, i) \in u \times (I \setminus I_u)\}$.

Then we can order $\hat{R}_u$ as $\hat{r}_{u, i_0} \geq \hat{r}_{u, i_1} \geq ... \geq \hat{r}_{u, i_k}$ and return the items $i \in (I \setminus I_u)$ associated with the *k*-largest predicted ratings $\hat{r}_{u, i}$.  


### Best-item

Can be seen as a particular case of top-$K$, with $K = 1$.


## 2.1 Non-personalized RS  

In this case we consider users only as providers of ratings, but **we ignore users' preferences when providing the recommendation**.  

We use this approach **only when we lack information about the preferences of the specific user we are providing a recommendation**.  

The rationale is that a generic user also likes something that is liked by many users.

* **Most-rated**: Check which elements are greater than zero (`R>0`), sum along the columns (`axis=0`) and **return the indices** of the top-$N$ most-rated items (`np.negative().argsort()[:n]`).  


* **Most-rated above threshold**: Check which elements are greater than threshold (`R>threshold`), sum along the columns (`axis=0`) and **return the indices** of the top-$N$ most-rated items (`np.negative().argsort()[:n]`).  


* **Higher average-rating**: Replace zeros by NaN, compute mean along the columns (`axis=0`) and **return the indices** of the top-$N$.


# 3 Implementation

## 3.1 Compressed Sparse Row (CSR) 

[numpy csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

The Compressed Sparse Row (CSR), uses three arrays:
* `data`, the value vector containing all non-zero values in [row-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order)
* `indptr`, the index pointer indicates at which element of the value vector the row starts
* `indices`, contains the column indices (which column each of the values come from).


In [3]:
data = np.array([1, 0, 2, 1, 5, 0, 0, 2, 1]).reshape(3, 3)
data

array([[1, 0, 2],
       [1, 5, 0],
       [0, 2, 1]])

In [15]:
R = csr_matrix(data)
R

<3x3 sparse matrix of type '<class 'numpy.int32'>'
	with 6 stored elements in Compressed Sparse Row format>

In [8]:
R.data

array([1, 2, 1, 5, 2, 1], dtype=int32)

In [13]:
R[0,0]

1

In [14]:
R.nnz

6

## 3.2 Numpy arrays vs. matrices  

Numpy matrices are strictly 2-dimensional, while numpy arrays (ndarrays) are N-dimensional. **Matrix objects are a subclass of ndarray**, so they inherit all the attributes and methods of ndarrays.

The main advantage of numpy matrices is that they provide a convenient notation for matrix multiplication: if a and b are matrices, then a*b is their matrix product.

**Warning!!!** The output of some numpy functions (like `np.argsort()`over an axis) for a matrix may not be as expected!

In [27]:
R.todense()

matrix([[1, 0, 2],
        [1, 5, 0],
        [0, 2, 1]], dtype=int32)

In [28]:
R.toarray()

array([[1, 0, 2],
       [1, 5, 0],
       [0, 2, 1]], dtype=int32)

In [29]:
np.asarray(R.todense())

array([[1, 0, 2],
       [1, 5, 0],
       [0, 2, 1]], dtype=int32)

## From pandas DataFrame to scipy sparse

In [17]:
data_df = pd.DataFrame(data)
data_df

,0,1,2
0,1,0,2
1,1,5,0
2,0,2,1


In [18]:
R_from_df = csr_matrix(data_df.values)

In [22]:
(R_from_df != R).sum()

0